# Prueba de Hipótesis Estadística
## H0: Accuracy ≤ 0.5 (rendimiento al azar)
## H1: Accuracy > 0.5 (modelo es mejor que el azar)


In [ ]:
import numpy as np
from scipy.stats import binomtest
from pathlib import Path
import joblib
import pandas as pd

# Cargar modelo y datos de test
from src.model.trainer import load_model, split_train_test
from src.preprocessing.pipeline import load_data, load_preprocessing_pipeline, transform_data

# Cargar datos y modelo
X, y = load_data()
preprocessor = load_preprocessing_pipeline()
X_transformed = transform_data(X, preprocessor)
X_train, X_test, y_train, y_test = split_train_test(X_transformed, y)

model = load_model()

# Predecir en test set
y_pred = model.predict(X_test)
correct_predictions = np.sum(y_pred == y_test)
n_samples = len(y_test)
accuracy = correct_predictions / n_samples

print(f"Número de muestras: {n_samples}")
print(f"Predicciones correctas: {correct_predictions}")
print(f"Accuracy observado: {accuracy:.4f}")


In [ ]:
# Prueba binomial exacta
# H0: p ≤ 0.5 (accuracy no mejor que azar)
# H1: p > 0.5 (accuracy mejor que azar)

p_value = binomtest(
    k=correct_predictions,
    n=n_samples,
    p=0.5,
    alternative='greater'
).pvalue

print(f"P-value: {p_value:.6f}")
print(f"P-value (notación científica): {p_value:.2e}")

# Nivel de significancia
alpha = 0.05
decision = "Rechazar H0" if p_value < alpha else "No rechazar H0"

print(f"\nNivel de significancia (α): {alpha}")
print(f"Decisión: {decision}")
print(f"\nInterpretación:")
if p_value < alpha:
    print(f"✓ Rechazamos H0: El modelo es significativamente mejor que el azar (p < {alpha})")
else:
    print(f"✗ No podemos rechazar H0: No hay evidencia suficiente de que el modelo sea mejor que el azar")


## Explicación de Errores Tipo I y Tipo II

### Error Tipo I (α)
- **Definición**: Rechazar H0 cuando H0 es verdadera (falso positivo)
- **En este contexto**: Concluir que el modelo es mejor que el azar cuando en realidad no lo es
- **Probabilidad**: α = 0.05 (5% de probabilidad)
- **Consecuencias**: Implementar un modelo que no es mejor que el azar, desperdiciando recursos

### Error Tipo II (β)
- **Definición**: No rechazar H0 cuando H1 es verdadera (falso negativo)
- **En este contexto**: No detectar que el modelo es mejor que el azar cuando realmente lo es
- **Probabilidad**: Depende del tamaño del efecto (diferencia real entre accuracy y 0.5)
- **Consecuencias**: Descartar un modelo útil, perder oportunidades de negocio

### Potencia de la Prueba (1 - β)
- **Definición**: Probabilidad de rechazar H0 cuando H1 es verdadera
- **Interpretación**: Capacidad de detectar un efecto real cuando existe
